# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [38]:
from pyspark.sql import SparkSession

In [39]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [40]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [41]:
# It is a tab separated file
data = spark.read.csv("smsspamcollection/SMSSpamCollection",
                      inferSchema=True, sep='\t')
data.show(2)

+---+--------------------+
|_c0|                 _c1|
+---+--------------------+
|ham|Go until jurong p...|
|ham|Ok lar... Joking ...|
+---+--------------------+
only showing top 2 rows



In [42]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [43]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [44]:
from pyspark.sql.functions import length

In [45]:
# Finding length of mail
# Length could be an important feature to detect spam vs non-spam
data = data.withColumn('length',length(data['text']))

In [46]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [53]:
# Pretty Clear Difference
#data.groupby('class').agg({'length':'mean'}).show()
#data.groupby('class').agg(avg(length)).show()
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [49]:
data.agg({'length':'mean'}).show()

+-----------------+
|      avg(length)|
+-----------------+
|80.46232508073197|
+-----------------+



## Feature Transformations

In [54]:
from pyspark.ml.feature import (Tokenizer,StopWordsRemover, 
                                CountVectorizer,IDF,StringIndexer)

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [55]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [56]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [57]:
from pyspark.ml.classification import NaiveBayes

In [58]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [59]:
from pyspark.ml import Pipeline

In [60]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [61]:
# fit transform can be done in 1 step
cleaner = data_prep_pipe.fit(data)

In [62]:
clean_data = cleaner.transform(data)

In [63]:
clean_data.show(2)

+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|length|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|Go until jurong p...|   111|  0.0|[go, until, juron...|[go, jurong, poin...|(13423,[7,11,31,6...|(13423,[7,11,31,6...|(13424,[7,11,31,6...|
|  ham|Ok lar... Joking ...|    29|  0.0|[ok, lar..., joki...|[ok, lar..., joki...|(13423,[0,24,297,...|(13423,[0,24,297,...|(13424,[0,24,297,...|
+-----+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



### Training and Evaluation!

In [64]:
clean_data = clean_data.select(['label','features'])

In [65]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [66]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [67]:
spam_predictor = nb.fit(training)

In [68]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [69]:
test_results = spam_predictor.transform(testing)

In [70]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,41,...|[-1089.9233011477...|[1.0,2.3634149754...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-819.98469683102...|[1.0,1.0999397950...|       0.0|
|  0.0|(13424,[0,1,7,8,1...|[-870.06173776008...|[1.0,2.5024112043...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-661.75983009742...|[1.0,5.1515912130...|       0.0|
|  0.0|(13424,[0,1,18,20...|[-866.92307281196...|[1.0,5.7627898973...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1469.3724143885...|[0.99999999999996...|       0.0|
|  0.0|(13424,[0,1,30,12...|[-596.49419056762...|[1.0,1.3085951152...|       0.0|
|  0.0|(13424,[0,1,46,17...|[-1137.8073262726...|[1.33398701785386...|       1.0|
|  0.0|(13424,[0,1,416,6...|[-315.40369754320...|[0.99999999935294...|       0.0|
|  0.0|(13424,[0

In [71]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [72]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy') 
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9186800235710076
